In [31]:
import os
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from sklearn.model_selection import train_test_split

In [3]:
def get_dataframe(language):
  # Set the directory path
  directory_path = 'C:\Research\labeled_features\{}'.format(language)

  # Create an empty list to store the dataframes
  dataframes = []

  # Iterate through all the files in the directory
  for file in os.listdir(directory_path):
    # Check if the file is a CSV file
    if file.endswith('.csv'):
      # Read the CSV file into a Pandas dataframe
      df = pd.read_csv(os.path.join(directory_path, file))

      # Drop rows where the 'label' column is not what was expected
      df = df[df['label'].isin([0, 1, 2, 3])]

      # Append the dataframe to the list
      dataframes.append(df)

  # Concatenate all the dataframes into a single dataframe
  df_all = pd.concat(dataframes)

  # Model was predicting negative values, so I had to remove the negative values
  df_all = df_all[df_all['label'].isin([0, 1, 2, 3])]

  # Add column with the index of each row, reset the indices
  df_all = df_all.reset_index(drop=True)

  # Create a list where at each index is the sample_id of that row
  sample_ids = df_all['sample_id'].tolist()

  # Drop the 'sample_id' column
  return df_all.drop(columns=['sample_id'])

In [4]:
# Get the dataframe
df_english = get_dataframe('EN')

In [8]:
def get_predictions(model, X_test):
    # Make predictions on the testing data
    unrounded_predictions = model.predict(X_test)

    # Round the predictions to the nearest integer
    predictions = np.round(unrounded_predictions)

    # Make all negative predictions 0
    predictions[predictions < 0] = 0

    return unrounded_predictions, predictions

In [9]:
def plot_conf_matrix(y_test, predictions, language_model, language_test):
    # Calculate the confusion matrix
    cm = confusion_matrix(y_test, predictions)

    # Convert the confusion matrix to a Pandas dataframe
    cm_df = pd.DataFrame(cm)#, index=['true 0', 'true 1', 'true 2', 'true 3'], columns=['pred 0', 'pred 1', 'pred 2', 'pred 3'])

    # Create a heatmap of the confusion matrix
    fig = px.imshow(cm_df, title='Confusion Matrix', text_auto=True)
    fig.update_layout(title='Confusion Matrix Predicted with {} model, on {} data'.format(language_model, language_test), xaxis_title='Predicted', yaxis_title='Expected')
    fig.show()

In [10]:
def plot_correlations(df_all, language_model, language_test):
    # Calculate the correlations between the columns and the label
    correlations = df_all.corr()['label'].iloc[:-1]

    # Create a scatter plot of the correlations
    fig = px.scatter(x=correlations.index, y=correlations, title='Correlations')
    fig.update_layout(title='Correlations between features and label, predicted with {} model, on {} data'.format(language_model, language_test), xaxis_title='Feature', yaxis_title='Correlation')
    fig.show()

    # Create a dataframe from the correlations
    correlations_df = pd.DataFrame(correlations)

    # correlations_df.style.background_gradient(cmap ='viridis')\
    # .set_properties(**{'font-size': '20px'})

In [11]:
def compute_correlation(unrounded_predictions, y_test):
    expected_predicted_df = pd.DataFrame()
    expected_predicted_df['expected'] = y_test
    expected_predicted_df['predicted'] = unrounded_predictions
    corr = expected_predicted_df.corr()['predicted'].iloc[0]
    return expected_predicted_df, corr

In [12]:
def plot_expected_vs_predicted(expected_predicted_df, language_model, language_test):
  fig = go.Figure()

  labels = expected_predicted_df['expected'].unique()

  for label in labels:
    fig.add_trace(go.Violin(
    x=expected_predicted_df['expected'][expected_predicted_df['expected'] == label],
    y=expected_predicted_df['predicted'][expected_predicted_df['expected'] == label],
    name=label,
    box_visible=True,
    meanline_visible=False,
    ))

  fig.update_layout(title='Expected vs Predicted, predicted with {} model, on {} data'.format(language_model, language_test), xaxis_title='Expected', yaxis_title='Predicted')
  fig.show()

In [39]:
X = df_english.iloc[:, :-1]
y = df_english['label']

y = to_categorical(y) # one-hot encode the labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the model
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(61, 1)))
model.add(Conv1D(32, kernel_size=3, activation='relu'))
model.add(Conv1D(16, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', test_acc)

Epoch 1/50
3123/3123 [==============================] - 25s 8ms/step - loss: 1.2134 - accuracy: 0.4035 - val_loss: 1.2004 - val_accuracy: 0.4275
Epoch 2/50
3123/3123 [==============================] - 27s 9ms/step - loss: 1.1948 - accuracy: 0.4268 - val_loss: 1.1853 - val_accuracy: 0.4448
Epoch 3/50
3123/3123 [==============================] - 28s 9ms/step - loss: 1.1820 - accuracy: 0.4388 - val_loss: 1.1708 - val_accuracy: 0.4555
Epoch 4/50
3123/3123 [==============================] - 27s 9ms/step - loss: 1.1700 - accuracy: 0.4484 - val_loss: 1.1608 - val_accuracy: 0.4624
Epoch 5/50
3123/3123 [==============================] - 30s 10ms/step - loss: 1.1594 - accuracy: 0.4578 - val_loss: 1.1470 - val_accuracy: 0.4725
Epoch 6/50
3123/3123 [==============================] - 29s 9ms/step - loss: 1.1483 - accuracy: 0.4639 - val_loss: 1.1336 - val_accuracy: 0.4752
Epoch 7/50
3123/3123 [==============================] - 30s 9ms/step - loss: 1.1383 - accuracy: 0.4661 - val_loss: 1.1221 - val_a

In [40]:
language_model = 'EN'
language_test = 'EN'

# Make predictions on the test data
y_pred = model.predict(X_test)

# Convert the predictions to the class with the highest probability
y_pred_class = np.argmax(y_pred, axis=1)

# Convert the true labels to the class format
y_test_class = np.argmax(y_test, axis=1)

# Compute the confusion matrix
conf_mat = confusion_matrix(y_test_class, y_pred_class)

# Create a heatmap of the confusion matrix
fig = px.imshow(conf_mat, title='Confusion Matrix', text_auto=True)
fig.update_layout(title='Confusion Matrix Predicted with {} model, on {} data'.format(language_model, language_test), xaxis_title='Predicted', yaxis_title='Expected')
fig.show()

781/781 [==============================] - 5s 7ms/step


In [41]:
# Calculate what percentage of the predictions are correct
accuracy = accuracy_score(y_test_class, y_pred_class)
print('Accuracy: {:.2f}%'.format(accuracy * 100))

# Calculate for each of the expected classes, what percentage of the predictions are correct
correct_percentage_0 = conf_mat[0, 0] / np.sum(conf_mat[0, :]) # 0
correct_percentage_1 = conf_mat[1, 1] / np.sum(conf_mat[1, :]) # 1
correct_percentage_2 = conf_mat[2, 2] / np.sum(conf_mat[2, :]) # 2
correct_percentage_3 = conf_mat[3, 3] / np.sum(conf_mat[3, :]) # 3

print('Correct percentage for class 0: {:.2f}%'.format(correct_percentage_0 * 100))
print('Correct percentage for class 1: {:.2f}%'.format(correct_percentage_1 * 100))
print('Correct percentage for class 2: {:.2f}%'.format(correct_percentage_2 * 100))
print('Correct percentage for class 3: {:.2f}%'.format(correct_percentage_3 * 100))

Accuracy: 59.53%
Correct percentage for class 0: 58.10%
Correct percentage for class 1: 80.49%
Correct percentage for class 2: 34.76%
Correct percentage for class 3: 20.47%
